In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import inegi_key #inegi token
from config import coneval_key #coneval
from config import gkey #google key

In [2]:


url = "https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr"
#declare parameters

EntFed = "15" #entidad federativa
Mpo    = "0" #municipio dentro de la entidad federativa
Loc    = "0" #loc dentro del mpo
AGEB   = "0" #AGEB no sé que es AGEB
Mnz    = "0" #manzana tres dígitos
Sector = "0" # Sector de la actividad económica (ej. 46 )
Subsec = "0" # Subsector de la actividad económica (ej. 464 )
Rama   = "0" #Rama de la actividad económica (ej. 4641)
Clase  = "0" #Clase de la actividad económica (ej. 464112)
Nomest = "0" # Nombre del establecimiento
Regini = "1" #Número de registro a partir del cuál se mostrarán los resultados de la búsqueda. 
Regfin = "25" #Número de registro final que se mostrará en resultados de la búsqueda. 
Id     = "0" #Clave única del establecimiento.
Estr   = "0" # Clave de un dígito del estrato. Para incluir todos los tamaños se especifica 0.
                #1. Para incluir de 0 a 5 personas.
                #2. Para incluir de 6 a 10 personas.
                #3. Para incluir de 11 a 30 personas.
                #4. Para incluir de 31 a 50 personas.
                #5. Para incluir de 51 a 100 personas.
                #6. Para incluir de 101 a 250 personas.
                #7. Para incluir de 251 y más person

#Ejemplo
#https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr/01/0/0/0/0/0/0/0/0/oxxo/1/15/0/1/[Aquí va tu token]
#https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr/01/0/0/0/0/0/0/0/0/oxxo/1/15/0/1/[8455939b-9726-bb8c-c70d-681609b40110]
#https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr/01/0/0/0/0/0/0/0/0/oxxo/1/15/0/1/[5537bc61-be6f-dddf-a592-e3cd4417e515]


urlconsulta = url + "/" + EntFed + "/" + Mpo + "/" + Loc + "/" + AGEB + "/" + Mnz + "/" + Sector + "/" + Subsec + "/" + Rama + "/" + Clase + "/" + Nomest + "/" + Regini + "/" + Regfin + "/" + Id + "/" + Estr + "/" + "[" + inegi_key + "]"

print(urlconsulta)
print(inegi_key)

https://www.inegi.org.mx/app/api/denue/v1/consulta/BuscarAreaActEstr/15/0/0/0/0/0/0/0/0/0/1/25/0/0/[8455939b-9726-bb8c-c70d-681609b40110]
8455939b-9726-bb8c-c70d-681609b40110
